In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from openai import OpenAI
import numpy as np
import os
import json

from bs4 import BeautifulSoup, NavigableString, CData, Tag
from io import BytesIO
import PyPDF2


In [1]:
import certifi
certifi.where()

'C:\\Users\\14102\\anaconda3\\envs\\crmm_ai\\Lib\\site-packages\\certifi\\cacert.pem'

In [2]:
# GPT Utils
def show_json(obj):
    display(json.loads(obj.model_dump_json()))


# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# HTML Parsing Funcs
def url_inline_text(soup):
    # Remove script and style elements
    for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()
    
    # # Remove comment elements
    # for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
    #     comment.extract()
    
    # Replace links with text and URL
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('javascript') or len(a['href']) < 2:
            a.decompose()
        else:
            a.replace_with(f"{a.get_text()} ({a['href']})")

    return soup
    

def get_shadow_root_text(driver, wait) -> list:
    # Augment HTML with text in shadow DOM
    shdw_dom_text = []
    all_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*')))
    for el in all_elements:
        try:
            if driver.execute_script('return arguments[0].shadowRoot', el):
                
                shdw_el = driver.execute_script(f"return document.querySelector('{el.tag_name}').shadowRoot.querySelector('div')")
                frame_soup = url_inline_text(BeautifulSoup(shdw_el.get_attribute('innerHTML'), 'html.parser'))
                shdw_dom_text.append(frame_soup.get_text(separator=' | '))
                
        except Exception as e:
            pass

    return shdw_dom_text


def get_full_dom_text(driver, wait) -> str:
    # Get HTML from main DOM
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    soup = url_inline_text(soup)
    # Get text, excluding any residual JavaScript or CSS
    main_html_text = soup.get_text(separator=' | ', strip=True)
    
    # Get any text from shadow root
    shadow_html_text_list = get_shadow_root_text(driver, wait)
    
    full_html = '|'.join([main_html_text] + shadow_html_text_list)

    return full_html

In [4]:
client = OpenAI()


# ast_file = client.files.retrieve('file-CClmpePzI1WIfdeDgw9bBvn6')
# Create Assistant
# assistant = client.beta.assistants.create(
#     name="Climate Resiliency Analyst",
#     description="You are an analyst responsible for judging how prepared infrastructure providers are for climate change in the context of the Climate Resilience Maturity Model",
#     instructions="Using information from the file 'Climate Resilience Maturity Model.txt', Evaluate text and URLs from given HTML on providers' websites and judge them in regards to the six capabilities defined in the Climate Risk Maturity Model. For each capability, provide a summary or reference to any relevant information, or return, 'no information' if no information is relevant",
#     model="gpt-4-turbo-preview",
#     tools=[{"type": "retrieval"}],
#     file_ids=[ast_file.id]
# )

assistant = client.beta.assistants.retrieve('asst_sOexDTYU2w6P21lwcwbyRVI7')

In [3]:
# Create web driver object
options = webdriver.ChromeOptions()
driver = webdriver.Chrome('C:\\Users\\14102\\Brown\\Internships\\INL\\CRMM_Scraping\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe', options=options)
# Method to navigate to provided URL
actions = ActionChains(driver)
wait = WebDriverWait(driver, 20)


In [4]:
driver.get('https://www.bge.com/safety-community/environment/our-commitment')

In [5]:
full_text = get_full_dom_text(driver=driver, wait=wait)

In [6]:
full_text = full_text.replace('\xa0', '')
full_text

'Our Commitment | BGE - An Exelon Company | Due to technical issues, currently we are only able to provide eligible bill assistance recommendations through the Assistance Finder if you go through the process as a guest. Our teams are working on resolving this and we apologize for any inconvenience. Please use this link | Assistance Finder (bge.com) (https://secure.bge.com/assistance/finder/account-type) | to evaluate assistance options you may be eligible for. | Our Commitment | Our Commitment | Our commitment to the environment extends beyond operating in compliance with applicable laws, regulations and requirements. We strive to not only act responsibly but also to serve in a leadership capacity in addressing the environmental challenges that impact our company and our stakeholders. | I can help you with: | Chat with Eli | Want a copy of this chat transcript? Enter your email address. | Submit | The chat transcript will be sent to | OK | Do you want to end the conversation? This will

## Integration Testing

In [11]:
# Create the thread for GPT interaction
dialogue = client.beta.threads.create()

# Initialize thread with user message. Insert inline HTML text/URLs in message as context
message = client.beta.threads.messages.create(
    thread_id=dialogue.id,
    role= "user",
    content=f"""Determine if there is any information in the provided HTML text and links that could be used to help inform the Climate Risk Maturity Model capabilities for Baltimore Gas and Electric. 
    
    HTML Text: {full_text}"""
)


message_2 = client.beta.threads.messages.create(
    thread_id=dialogue.id,
    role= "user",
    content=f"""identify up to 5 links in the text from my last message which could lead to additional information related to the Climate Risk Maturity Model."""
)


In [22]:
# Run interaction with GPT (Kicks off request)

run = client.beta.threads.runs.create(
  thread_id=dialogue.id,
  assistant_id=assistant.id)

In [25]:
# Check Response
run_status = client.beta.threads.runs.retrieve(
  thread_id=dialogue.id,
  run_id=run.id
)
show_json(run_status)

{'id': 'run_RloIBk7NIMmSJbxbpxbyDVo2',
 'assistant_id': 'asst_sOexDTYU2w6P21lwcwbyRVI7',
 'cancelled_at': None,
 'completed_at': 1706746008,
 'created_at': 1706745975,
 'expires_at': None,
 'failed_at': None,
 'file_ids': ['file-CClmpePzI1WIfdeDgw9bBvn6'],
 'instructions': "Using information from the file 'Climate Resilience Maturity Model.txt', Evaluate text and URLs from given HTML on providers' websites and judge them in regards to the six capabilities defined in the Climate Risk Maturity Model. For each capability, provide a summary or reference to any relevant information, or return, 'no information' if no information is relevant",
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-turbo-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1706745976,
 'status': 'completed',
 'thread_id': 'thread_jW8A2nxzD9d0KqbYoC8tP05H',
 'tools': [{'type': 'retrieval'}],
 'usage': {'prompt_tokens': 6068,
  'completion_tokens': 608,
  'total_tokens': 6676}}

In [26]:
# Display result

messages = client.beta.threads.messages.list(
  thread_id=dialogue.id
, order='asc')
pretty_print(messages)

# Messages
user: Determine if there is any information in the provided HTML text and links that could be used to help inform the Climate Risk Maturity Model capabilities for Baltimore Gas and Electric. 
    
    HTML Text: Our Commitment | BGE - An Exelon Company | Due to technical issues, currently we are only able to provide eligible bill assistance recommendations through the Assistance Finder if you go through the process as a guest. Our teams are working on resolving this and we apologize for any inconvenience. Please use this link | Assistance Finder (bge.com) (https://secure.bge.com/assistance/finder/account-type) | to evaluate assistance options you may be eligible for. | Customer bills will increase effective with usage starting in January 2024 as authorized by the Maryland Public Service Commission. The average residential electric bill will increase by $1.76 per month and the average residential natural gas bill will increase by $6.43 per month. See | bge.com/myp  (https://w

In [20]:
# Follow-up question for the links
message_2 = client.beta.threads.messages.create(
    thread_id=dialogue.id,
    role= "user",
    content=f"""identify up to 5 links in the text from my last message which could lead to additional information related to the Climate Risk Maturity Model. Only return the full URL and related capabilities in a bulleted list."""
)

In [21]:
# Run again
run = client.beta.threads.runs.create(
  thread_id=dialogue.id,
  assistant_id=assistant.id)

{'id': 'thread_jW8A2nxzD9d0KqbYoC8tP05H',
 'created_at': 1706745615,
 'metadata': {},
 'object': 'thread'}

In [ ]:
# Display new result

messages = client.beta.threads.messages.list(
  thread_id=dialogue.id
, order='asc')
pretty_print(messages)